In [62]:
import xarray as xr
import pandas as pd
import numpy as np 
import dask as da
import shapely as shp
from scipy.interpolate import interp1d
from pathlib import Path

In [13]:
path = Path.cwd().parent / 'data' / 'in'
path

PosixPath('/home/hmack/Development/onehealth-model-backend/data/in')

# get data

In [22]:
data = xr.open_dataset(path / "ERA5land_global_t2m_dailyStats_mean_01Deg_2024_08_data.nc")
data

<xarray.Dataset> Size: 804MB
Dimensions:     (valid_time: 31, latitude: 1801, longitude: 3600)
Coordinates:
    number      int64 8B ...
  * latitude    (latitude) float64 14kB 90.0 89.9 89.8 ... -89.8 -89.9 -90.0
  * valid_time  (valid_time) datetime64[ns] 248B 2024-08-01 ... 2024-08-31
  * longitude   (longitude) float64 29kB -179.9 -179.8 -179.7 ... 179.9 180.0
Data variables:
    t2m         (valid_time, latitude, longitude) float32 804MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-07T15:15 GRIB to CDM+CF via cfgrib-0.9.1...

In [23]:
temp_data = xr.Dataset(pd.read_csv(path / "R0_pip_stats.csv")) 
temp_data

<xarray.Dataset> Size: 14kB
Dimensions:      (dim_0: 451)
Coordinates:
  * dim_0        (dim_0) int64 4kB 0 1 2 3 4 5 6 ... 444 445 446 447 448 449 450
Data variables:
    Temperature  (dim_0) float64 4kB 0.0 0.1 0.2 0.3 0.4 ... 44.7 44.8 44.9 45.0
    Median_R0    (dim_0) float64 4kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Species      (dim_0) object 4kB 'pip.' 'pip.' 'pip.' ... 'pip.' 'pip.'

# interpolation function

In [113]:
def process_r0(temp_data:xr.Dataset, min_temp:np.float64 = 0.0, max_temp:np.float64 = 45.0, step:np.float64 = 0.1) -> xr.Dataset:
    temps = np.arange(min_temp, max_temp+step, step)

    valid = (~np.isnan(temp_data.Median_R0) & (temp_data.Temperature >= min_temp) & (temp_data.Temperature <= max_temp )).values

    interpolated = np.interp(
        temps, 
        temp_data.Temperature[valid].values, 
        temp_data.Median_R0[valid].values, 
        left=np.nan, 
        right=np.nan
    )

    return xr.Dataset(
        data_vars={
            'Temperature': ('Temperature', temps),
            'Median_R0': ('Temperature', interpolated)
        },
    )



In [121]:
process_r0(temp_data)

<xarray.Dataset> Size: 7kB
Dimensions:      (Temperature: 451)
Coordinates:
  * Temperature  (Temperature) float64 4kB 0.0 0.1 0.2 0.3 ... 44.8 44.9 45.0
Data variables:
    Median_R0    (Temperature) float64 4kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

# apply the function to the data